<a href="https://colab.research.google.com/github/slvsccp/Uninove/blob/main/twitter_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas necessárias

In [ ]:
import tweepy
import pandas as pd
from datetime import datetime
from textblob import TextBlob
import matplotlib.pyplot as plt
%matplotlib inline

# Dados da autenticação

In [ ]:
# dados das chaves
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

# Autenticando no Twitter via API

In [ ]:
# realizando a autenticação
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=5, retry_delay=10)

# Métodos para realizar a pesquisa por palavra ou frase

+ *Polarity* (polaridade) - é um valor contínuo que varia entre -1.0 e 1.0 e se refere a 100% negativo ou 100% positivo

+ *Subjectivity* (subjetividade) - é uma valor contínuo entre 0.0 e 1.0 onde 100% objetivo e 100% subjetivo

In [ ]:
count = 50 # total de registros que queremos recuperar
lang = 'pt' # idioma
keyword = "'Uninove' OR 'Uni9' OR 'uni9'" # o que queremos procurar

# configurar a requisição ao twitter
tweets = api.search(
    q = keyword, # palavra ou frase para pesquisa
    rpp = count, # números de registros a serem retornados
    result_type = 'mixed', # tipo de resultado que queremos (texto completo da mensagem)
    since = datetime(2020, 6, 30, 0, 0, 0).date(), # data a partir da qual queremos pesquisar
    lang = lang # idioma da pesquisa
)

dados = [] # vamos guardar os dados de retorno
sentimento = [] # vamos guardar o sentimento de cada frase (postagem)

for tweet in tweets: # percorrer cada uma das mensagens retornadas
    frase = TextBlob(tweet.text) # colocamos cada mensagem retornada na variável para tratamento

    # vamos traduzir automaticamente para o inglês, a fim de usar a biblioteca TextBlob para veriricar o sentimento
    if frase.detect_language() != 'en':
        traducao = TextBlob(str(frase.translate(to='en')))
        dados.append(
            [tweet.user.name, tweet.text, traducao.sentiment[0], traducao.sentiment[1]]
        )
    else:
        dados.append(
            [tweet.user.name, tweet.text, frase.sentiment[0], frase.sentiment[1]]
        )

    # verificar os sentimentos
    if traducao.sentiment[0] > 0:
        sentimento.append('Positivo')
    elif traducao.sentiment[0] < 0:
        sentimento.append('Negativo')
    else:
        sentimento.append('Indiferente')

tweets_df = pd.DataFrame(dados, columns=['usuario', 'tweet', 'sentimento', 'polaridade'])
tweets_df['status'] = sentimento # agregamos a coluna status ao DataFrame

# ordenar os dados por importância do sentimento
tweets_df.sort_values(by='sentimento', ascending=False)

In [ ]:
# mostra todos os registros da coluna status
tweets_df.status 

In [ ]:
# Contando as ocorrências dos "status"
tweets_df.status.value_counts() 

In [ ]:
# Plotagem do gráfico dos "status"
plt.figure(1, figsize=(20, 6))
plt.title("Status das mensagens (sentimento)")
tweets_df.status.value_counts().plot.bar()
plt.show()

In [ ]:
# Percorrendo linha a linha o dataset com iterrows() e mostrando o resultado
for index, row in tweets_df.iterrows():
    print(row.usuario, row.tweet, row.status)